In [19]:
import pandas as pd
import os
from datetime import datetime

In [20]:
events = []


def parse_time(time_string):
    return datetime.strptime(time_string, "%M:%S,%f")


def event(player, event_name, time_string):
    return {"player": player, "event": event_name, "timestamp": parse_time(time_string)}


def player_id(player_string):
    return int(player_string.replace("<Files\\\\Player ", "").split(">")[0])

In [21]:
file = open('../Codes/Chamber.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "chamber_enter", time_strings[0]))
        events.append(event(current_player, "chamber_exit", time_strings[1]))

In [22]:
file = open('../Codes/Death.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "death", time_strings[0]))

In [23]:
file = open('../Codes/Meg Encounter.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "boss_fight_begin", time_strings[0]))
        events.append(event(current_player, "boss_fight_end", time_strings[1]))

In [24]:
file = open('../Codes/Run.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "run_begin", time_strings[0]))
        events.append(event(current_player, "run_end", time_strings[1]))

In [25]:
dataset = pd.DataFrame(events).sort_values(by=["player", "timestamp"])
dataset["chamber"] = None
dataset["attempt"] = None

attempt = 0
in_run = False
chamber = None
player = 1
death = False
for index, row in dataset.iterrows():
    if player < row["player"]:
        player += 1
        attempt = 0
        chamber = None

    if row["event"] == "chamber_enter":
        if in_run is False:
            in_run = True
            chamber = 1
            attempt += 1
            death = False
        elif chamber is None:
            chamber = 1
        else:
            chamber += 1

    if row["event"] == "run_begin":
        if in_run is False:
            in_run = True
            death = False
            chamber = None
            attempt += 1

    if row["event"] == "death":
        death = True

    if row["event"] == "chamber_exit" and chamber is None:
        chamber = 1

    dataset.at[index, 'chamber'] = chamber
    dataset.at[index, 'attempt'] = attempt

    if row["event"] == "run_end":
        in_run = False

dataset.to_csv("../Processed/game_events.csv", index=False, date_format="%M:%S:%f")

In [26]:
def player_stats(player):
    return pd.Series({"mean_run_time": (player["timestamp"].max() - player["timestamp"].min()).total_seconds()/player["attempt"].max(),
                      "total_chambers": player["event"][player["event"] == "chamber_enter"].count(),
                      "max_chamber": player["chamber"].max(),
                      "mean_chamber_per_run": player["event"][player["event"] == "chamber_enter"].count() / player[
                          "attempt"].max(),
                      "number_of_runs": player["attempt"].max(),
                      "boss_encounters": player["event"][player["event"] == "boss_fight_begin"].count()
                      })


dataset.groupby(["player"]).apply(player_stats).reset_index().to_csv("../Processed/player_stats.csv", index=False,
                                                                     date_format="%M:%S:%f")

In [27]:
def run_stats(attempt):
    return pd.Series({"run_start": attempt["timestamp"].min(), "run_end": attempt["timestamp"].max(),
                      "time": (attempt["timestamp"].max() - attempt["timestamp"].min()).total_seconds(),
                      "chambers": attempt["chamber"].max(),
                      "boss_encounters": attempt["event"][attempt["event"] == "boss_fight_begin"].count()
                      })


dataset.groupby(["player", "attempt"]).apply(run_stats).reset_index().to_csv("../Processed/run_stats.csv", index=False,
                                                                             date_format="%M:%S:%f")